In [3]:
import numpy as np, random, math, operator, pandas as pd, matplotlib.pyplot as plt

In [4]:
random.seed(42)

# Setup

La manière dont le code fonctionne:

Il y a la classe `City` qui est en gros juste les coordonnées (x,y) de sa position, ainsi qu'une méthode pour avoir la distance.

La classe Fitness permet de, étant donné une chemin (une `route`), d'avoir la distance de ce chemin grâce à la méthode `routeDistance`, et la "Fitness" de ce chemin par la méthode `routeFitness` (qui est juste l'inverse de la distance).

Ensuite chaque agent est constitué d'un ensemble de méthodes qui permettent d'implémenter sa fonction de recherche, comme le Recuit Simulé ou bien l'algo génétique. Ces fonctions sont au final rassemblées dans la fonction `_step` qui fait une étape de l'algorithme, et la fonction `search` qui fait l'ensemble de la recherche. 
Les paramètres à donner à ces agents, tant à l'initialisation qu'à la recherche par `search`, sont variables selon l'algo utilisé par l'agent.
En particulier, un Agent est initializé avec l'information de combien d'appels à `_step` un appel à `search` va occasionner, dans `self.generations`.

In [5]:
class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((xDis ** 2) + (yDis ** 2))
        return distance
    
    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) + ")"

In [6]:
class Fitness:
    def __init__(self, route):
        self.route = route
        self.distance = 0
        self.fitness= 0.0
    
    def routeDistance(self):
        if self.distance ==0:
            pathDistance = 0
            for i in range(0, len(self.route)):
                fromCity = self.route[i]
                toCity = None
                if i + 1 < len(self.route):
                    toCity = self.route[i + 1]
                else:
                    toCity = self.route[0]
                pathDistance += fromCity.distance(toCity)
            self.distance = pathDistance
        return self.distance
    
    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.routeDistance())
        return self.fitness
    
    


# Agent superclass

In [121]:
class SearchAgent():
    """
    Implements the `update` method common to all search agents. 
    What this method does is, according to the current agent status, decide
    what to do. 
    
    Status:
        IDLE: waits (firt state, will exit at first update() execution)
        OPTM: optimizing (finding a better solution. If stuck in local optima
              the agent will go to COMP status and wait for another agent 
              to find a better optima)
        COMP: best local solution, comparison with other agents. Will only
              exit if another agent finds a better solution
        
    """
    def __init__(self):
        self.status = "IDLE"
        self.route = None
        self.routeDistance = None
        
    def search():
        pass
    
    def update(self, best_other_route=None, best_other_routeDistance=None):
        """
        Does an update of the agent, according to its status
        
        Input:
            Either nothing, or
            the best current route and its distance
        Returns:
            Agent's status, current best route, current best distance
        """
        
        if self.status == "IDLE":
            self.route, self.routeDistance = self.search()
            self.status == "OPTM"
            return self.status, self.route, self.routeDistance
            
        elif self.status == "OPTM":
            route, routeDistance = self.search()
            # if the route distance didn't change much from last udpate() call
            if (routeDistance > 0.98 * self.routeDistance) and (routeDistance < 1.02 * self.routeDistance):
                    self.status = "COMP"
                    return self.status, self.route, self.routeDistance
                
            self.route = route
            self.routeDistance = routeDistance
            return self.status, self.route, self.routeDistance
        
        elif self.status == "COMP":
            # if the current best route is provided
            if best_other_route is not None:
                # if another agent found a better route, take that one and 
                # goes back to optimization
                if best_other_routeDistance < self.routeDistance:
                    self.route = best_other_route
                    self.routeDistance = best_other_routeDistance
                    self.status = "OPTM"
                    
                    return self.status, self.route, self.routeDistance
                # else just wait for another agent to improve that solution
                else:
                    return self.status, self.route, self.routeDistance
            # this won't happen after the first iteration of the CoopAgent
            else:
                return self.status, self.route, self.routeDistance

    
                
            
        

# GAAGent

In [122]:
class GAAgent(SearchAgent):
    """
    Hyperparameters: 
        intensification: eliteSize
        diversification: popSize, mutationRate
    """
    
    def __init__(self, popSize, eliteSize, mutationRate, population, generations):
        super().__init__()
        self.popSize = popSize
        self.eliteSize = eliteSize
        self.mutationRate = mutationRate
        self.population = population
        self.generations = generations
    
    
    def createRoute(self, cityList):
        return random.sample(cityList, len(cityList))

    def initialPopulation(self, popSize, cityList):
        return [self.createRoute(cityList) for i in range(popSize)]

    def rankRoutes(self, population):
        fitnessResults = {i:Fitness(population[i]).routeFitness() for i in range(len(population))}
        return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)

    def selection(self, popRanked, eliteSize):
        df = pd.DataFrame(np.array(popRanked), columns=["Index","Fitness"])
        df['cum_sum'] = df.Fitness.cumsum()
        df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()
        
        selectionResults = [popRanked[i][0] for i in range(eliteSize)]
        for i in range(0, len(popRanked) - eliteSize):
            pick = 100*random.random()
            for i in range(0, len(popRanked)):
                if pick <= df.iat[i,3]:
                    selectionResults.append(popRanked[i][0])
                    break
                    
        return selectionResults


    def matingPool(self, population, selectionResults):
        matingpool = [population[selectionResults[i]] for i in range(len(selectionResults))]
        return matingpool


    def breed(self, parent1, parent2):
        child = []
        childP1 = []
        childP2 = []

        geneA = int(random.random() * len(parent1))
        geneB = int(random.random() * len(parent1))

        startGene = min(geneA, geneB)
        endGene = max(geneA, geneB)

        for i in range(startGene, endGene):
            childP1.append(parent1[i])

        childP2 = [item for item in parent2 if item not in childP1]

        child = childP1 + childP2
        return child

    def breedPopulation(self, matingpool, eliteSize):
        children = []
        length = len(matingpool) - eliteSize
        pool = random.sample(matingpool, len(matingpool))

        for i in range(0,eliteSize):
            children.append(matingpool[i])

        for i in range(0, length):
            child = self.breed(pool[i], pool[len(matingpool)-i-1])
            children.append(child)
        return children


    def mutate(self, individual, mutationRate):
        for swapped in range(len(individual)):
            if(random.random() < mutationRate):
                swapWith = int(random.random() * len(individual))

                city1 = individual[swapped]
                city2 = individual[swapWith]

                individual[swapped] = city2
                individual[swapWith] = city1
        return individual


    def mutatePopulation(self, population, mutationRate):
        mutatedPop = []

        for ind in range(0, len(population)):
            mutatedInd = self.mutate(population[ind], mutationRate)
            mutatedPop.append(mutatedInd)
        return mutatedPop

    def _step(self, currentGen, eliteSize, mutationRate):
        popRanked = self.rankRoutes(currentGen)
        selectionResults = self.selection(popRanked, eliteSize)
        matingpool = self.matingPool(currentGen, selectionResults)
        children = self.breedPopulation(matingpool, eliteSize)
        nextGeneration = self.mutatePopulation(children, mutationRate)
        return nextGeneration

    def search(self):
        
        pop = self.initialPopulation(self.popSize, self.population)
        for i in range(self.generations):
            pop = self._step(pop, self.eliteSize, self.mutationRate)
        bestRouteIndex = self.rankRoutes(pop)[0][0]
        bestRoute = pop[bestRouteIndex]
        
        return bestRoute, Fitness(bestRoute).routeDistance()

In [123]:
def create_cityList(size=25):
    random.seed(42)
    return [City(x=int(random.random() * 200), y=int(random.random() * 200)) for i in range(size)]

In [124]:
cityList = create_cityList(10)

In [125]:
cityList

[(127,5),
 (55,44),
 (147,135),
 (178,17),
 (84,5),
 (43,101),
 (5,39),
 (129,108),
 (44,117),
 (161,1)]

In [126]:
def test_GAA(route, print_route=False):

    GAA = GAAgent(popSize=100, eliteSize=20, mutationRate=0.01, population=cityList, generations=200)

    chemin, E = GAA.search()
    
    if print_route: print(route)

    print(E)

In [127]:
test_GAA(cityList, print_route=True)

[(127,5), (55,44), (147,135), (178,17), (84,5), (43,101), (5,39), (129,108), (44,117), (161,1)]
528.1085679727015


# RSAGent

In [128]:
class RSAgent(SearchAgent):
    """
    Hyperparameters: 
        t_init, t_min, coeff
    """
    
    def __init__(self, route_init, t_init, coeff, generations):
        super().__init__()
        self.n = len(route_init)
        self.route_init = route_init
        self.temp = t_init
        self.coeff = coeff
        self.generations = generations
    
    def _energy(self, route):
        """
        Compute energy of the route = route distance
        """
    
        return Fitness(route).routeDistance()
    
    def _step(self, route, E):
        """
        Does a step of the RS algorithm
        """
        
    
        sommet1 = random.randint(0, self.n-1)
        sommet2 = random.randint(0, self.n-1)

        route_b = route[:]

        temporaire = route_b[sommet1]
        route_b[sommet1] = route[sommet2]
        route_b[sommet2] = temporaire
        E_b = self._energy(route_b)

        if (E_b < E) or (random.random() < math.exp((E - E_b) / self.temp)):
            route = route_b
            E = E_b

        self.temp *= self.coeff
        
        return route, E
        
    
    
    def search(self):
        """

        """
        
        route = self.route_init
        E = self._energy(route)
        
        for i in range(self.generations):
            
            route, E = self._step(route, E)
            
        return route, E
        

In [129]:
def test_RSA(route, print_route = False):

    RSA = RSAgent(route, t_init = 100, coeff = 0.99, generations=500)
    
    route, E = RSA.search()
    
    if print_route: print(route)

    print(E)

In [130]:
test_RSA(cityList, print_route=True)

[(55,44), (5,39), (84,5), (127,5), (161,1), (178,17), (43,101), (44,117), (147,135), (129,108)]
646.7136789382805


# TabouAgent

In [131]:
class GAAgent():
    
    
    

SyntaxError: unexpected EOF while parsing (<ipython-input-131-9f020178e4c1>, line 4)

# CoopAgent

In [137]:
class CoopAgent():
    """
    This agent is the one that's creating and coordinating all the other agent
    work. 
    It first create the agents with various parameters, and then goes through
    cycles of updates to make the agents cooperate according to their status
    """
    
    def __init__(self, agents, nb_cycles):
        self.agents = agents
        self.nb_cycles = nb_cycles
        self.bestRoute = None
        self.bestRouteDistance = None
    
    
    def cycle(self, print_cycles=False):
        for cycle in range(self.nb_cycles):
            for agent in self.agents:
                if print_cycles: 
                    print(cycle)
                    print(agent)
                    print(self.bestRouteDistance)
                status, route, distance = agent.update(self.bestRoute, self.bestRouteDistance)
                if (not self.bestRouteDistance) or distance < self.bestRouteDistance:
                    self.bestRoute = route
                    self.bestRouteDistance = distance
                    
                
            
    
    

In [145]:
def test_CoopA(cityList):

    RSA1 = RSAgent(cityList, t_init = 100, coeff = 0.99, generations=500)    
    GAA1 = GAAgent(popSize=100, eliteSize=20, mutationRate=0.01, population=cityList, generations=100)
    GAA2 = GAAgent(popSize=100, eliteSize=30, mutationRate=0.02, population=cityList, generations=100)

    agents = [RSA1, GAA1, GAA2]
    
    CoopA = CoopAgent(agents, 20)
    
    CoopA.cycle(print_cycles=True)

In [143]:
cityList = create_cityList(50)

In [146]:
test_CoopA(cityList, )

0
None
0
2614.69193922451
0
2614.69193922451
1
2614.69193922451
1
2574.0914153757108
1
2574.0914153757108
2
2574.0914153757108
2
2534.4841052745574
2
2534.4841052745574
3
2534.4841052745574
3
2534.4841052745574
3
2430.5475057570343
4
2430.5475057570343
4
2430.5475057570343
4
2430.5475057570343
5
2430.5475057570343
5
2430.5475057570343
5
2430.5475057570343
6
2430.5475057570343
6
2430.5475057570343
6
2430.5475057570343
7
2430.5475057570343
7
2430.5475057570343
7
2430.5475057570343
8
2430.5475057570343
8
2409.896415107113
8
2409.896415107113
9
2409.896415107113
9
2409.896415107113
9
2409.896415107113
10
2409.896415107113
10
2409.896415107113
10
2409.896415107113
11
2409.896415107113
11
2409.896415107113
11
2397.3879916437754
12
2397.3879916437754
12
2397.3879916437754
12
2397.3879916437754
13
2397.3879916437754
13
2397.3879916437754
13
2397.3879916437754
14
2397.3879916437754
14
2397.3879916437754
14
2397.3879916437754
15
2397.3879916437754
15
2397.3879916437754
15
2397.3879916437754
16
2